### classify stackoverflow posts using tf-idf features, using one-vs-all SVM

- note: use the same tokenizer rules as Keras

In [11]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

In [2]:
SEED=np.random.randint(1,1000)
SEED

774

In [3]:
np.random.seed(seed=SEED)

In [5]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [6]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [8]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 'it',
 's',
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',


In [9]:
tokenized_texts = []

for seq in sequences:
    truncated_seq = seq[:MAX_SEQUENCE_LENGTH]
    tokenized_txt = " ".join([inverse_word_index[idx] for idx in truncated_seq])
    tokenized_texts.append(tokenized_txt)

In [10]:
data = tokenized_texts
indices = np.arange(len(data))
np.random.shuffle(indices)

labels_1 = to_categorical(np.asarray(labels_0))

data = [data[i] for i in indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * len(data))

X_train = data[:-num_validation_samples]
Y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_2[-num_validation_samples:]

len(X_train),len(X_val),Y_train.shape,Y_val.shape

(15998, 3999, (15998, 20), (3999, 20))

In [14]:
# good order (OVR just for the SVM, of course!)
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(svm.LinearSVC(),n_jobs=-1)),
])

parameters = [
    { 
          "clf__estimator__penalty": ["l2"],
          "clf__estimator__dual":[False,True],
          "clf__estimator__multi_class":["crammer_singer","ovr"],
          "clf__estimator__tol": [0.001],
          "clf__estimator__max_iter":[25,50,100,500],       
          "vect__max_features": [MAX_NB_WORDS]  
    },
    { 
          "clf__estimator__penalty": ["l1"],
          "clf__estimator__dual":[False],
          "clf__estimator__multi_class":["crammer_singer","ovr"],
          "clf__estimator__tol": [0.001],
          "clf__estimator__max_iter":[25,50,100,500],
          "vect__max_features": [MAX_NB_WORDS]
    }    
]

In [15]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(X_train,Y_train)
    
    Y_pred_train = clf.predict(X_train)    
    Y_pred_val = clf.predict(X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

CPU times: user 3.18 s, sys: 104 ms, total: 3.29 s
Wall time: 6.69 s
train accuracy: 0.9428678584823103
val accuracy: 0.735933983495874
grid: {'clf__estimator__max_iter': 25, 'clf__estimator__penalty': 'l2', 'clf__estimator__dual': False, 'vect__max_features': 20000, 'clf__estimator__tol': 0.001, 'clf__estimator__multi_class': 'crammer_singer'}

CPU times: user 3.08 s, sys: 104 ms, total: 3.18 s
Wall time: 4.36 s
train accuracy: 0.9556819602450306
val accuracy: 0.7029257314328582
grid: {'clf__estimator__max_iter': 25, 'clf__estimator__penalty': 'l2', 'clf__estimator__dual': False, 'vect__max_features': 20000, 'clf__estimator__tol': 0.001, 'clf__estimator__multi_class': 'ovr'}

CPU times: user 3.1 s, sys: 96 ms, total: 3.2 s
Wall time: 6.32 s
train accuracy: 0.9428678584823103
val accuracy: 0.735933983495874
grid: {'clf__estimator__max_iter': 50, 'clf__estimator__penalty': 'l2', 'clf__estimator__dual': False, 'vect__max_features': 20000, 'clf__estimator__tol': 0.001, 'clf__estimator__mu